In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())
# from pyspark.mllib.feature import HashingTF, IDF

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F   

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
#тестовые товары

test_schema = StructType(fields=[StructField("item",StringType()),
                                StructField("true_recoms",StringType())#MapType(StringType(), IntegerType()))
                                ])
ozon_train = spark.read.json("/labs/project02/ozon_train.txt",schema=test_schema)
ozon_train.registerTempTable('ozon_train')  

In [ ]:
ozon_train.toPandas().to_csv('train.csv')

In [ ]:
#тестовые товары

test_schema = StructType(fields=[StructField("item",StringType()),
                                ])
ozon_test = spark.read.json("/labs/project02/ozon_test.txt",schema=test_schema)
ozon_test.registerTempTable('ozon_test')  

In [ ]:
ozon_test.toPandas().to_csv('test.csv')

In [ ]:
spark.stop()

In [1]:
import re
from collections import defaultdict

import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import scipy
from tqdm import tqdm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.evaluation import auc_score

%matplotlib inline
pd.set_option('max_rows',200)
pd.set_option('max_columns',200)
import json

In [2]:
# catalogs.csv255 MB6 минут назад
# ozon_test.txt2.24 MB19 часов назад
# ozon_train.txt19.1 MB19 часов назад
# test.csv889 kBнесколько секунд назад
# train.csv16.4 MBнесколько секунд назад
# train_exploded.csv

In [8]:
# catalogs = pd.read_csv('catalogs.csv',dtype=str).drop('Unnamed: 0',axis=1)

In [5]:
# catalog_path = pd.read_csv('catalog_path.csv',dtype=str).drop('Unnamed: 0',axis=1)

In [2]:
train = pd.read_csv('train.csv',dtype=str).drop('Unnamed: 0',axis=1)
test = pd.read_csv('test.csv',dtype=str).drop('Unnamed: 0',axis=1)

In [3]:
train = train[train['true_recoms'].map(json.loads).map(lambda x: len(x.keys()))<200]
train['true_recoms'] = train['true_recoms'].map(json.loads)

In [5]:
rows = []
_ = train.apply(lambda row: [rows.append([row['item'], nn, row['true_recoms'][nn]]) 
                         for nn in row['true_recoms']], axis=1)

In [6]:
train_new = pd.DataFrame(rows, columns=['item','item_rec','count'])

In [9]:
dataset = Dataset()
dataset.fit(users = (x for x in train_new['item']),
            items = (x for x in train_new['item_rec'])
           )
# dataset.fit_partial(users = (x for x in train_new['item_rec']),
#             items = (x for x in train_new['item_rec'])
#            )

In [10]:
(interactions, weights) = dataset.build_interactions(((x[1]['item'], x[1]['item_rec'], x[1]['count'])
                                                      for x in train_new.iterrows()))

In [49]:
from lightfm import LightFM
from IPython.display import clear_output
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
model = LightFM(no_components=16, learning_rate=0.04, loss='warp', max_sampled=100, random_state=0)

from tqdm import tqdm_notebook

num_epochs = 50
for i in tqdm_notebook(range(num_epochs), total=num_epochs):
    model.fit_partial(
        interactions
    )

In [50]:
num_users, num_items = dataset.interactions_shape()
users_mapping, user_features_mapping, assets_mapping, asset_features_mapping = dataset.mapping()
print(len(users_mapping), len(user_features_mapping),len(assets_mapping), len(asset_features_mapping))
num_user_features = dataset.user_features_shape()
num_asset_features = dataset.item_features_shape()
print('Num user features: {} -> {}\nnum asset features: {} -> {}.'.format(num_user_features[1] - num_users, num_user_features[1], num_asset_features[1] - num_items, num_asset_features[1]))
print(len(user_features_mapping), len(asset_features_mapping))

142437 142437 337756 337756
Num user features: 0 -> 142437
num asset features: 0 -> 337756.
142437 337756


In [51]:
test.shape

(60956, 1)

In [52]:

with open('~/project02.txt', 'a') as f:
    for row in tqdm_notebook(test.iterrows()):
        obj = row[1]['item']
        if obj in asset_features_mapping:
            asset_features_mapping_inv = {j:i for i,j in asset_features_mapping.items()}
            def get_similar_tags(model, tag_id):
                # Define similarity as the cosine of the angle
                # between the tag latent vectors

                # Normalize the vectors to unit length
                tag_embeddings = (model.item_embeddings.T
                                  / np.linalg.norm(model.item_embeddings, axis=1)).T

                query_embedding = tag_embeddings[tag_id]
                similarity = np.dot(tag_embeddings, query_embedding)
                most_similar = np.argsort(-similarity)[1:101]

                return most_similar,(similarity)[most_similar]


            tag_id = asset_features_mapping[obj]
            similar,similarity = get_similar_tags(model, tag_id)

            similar = [asset_features_mapping_inv[x] for x in similar]

            f.write(json.dumps({"item":obj,"recoms":{i:float(j) for i, j in zip(similar,similarity)}})+'\n')
        else:
#             print(obj) 6242189
            f.write(json.dumps({"item":obj,"recoms":{}})+'\n')


In [54]:
# !cp project02.txt ~/project02.txt

In [ ]:
# def similar_items(item_id, item_features, model, N=10):
#     item_representations = item_features.dot(model.item_embeddings)

#     # Cosine similarity
#     scores = item_representations.dot(item_representations[item_id, :])
#     item_norms = np.linalg.norm(item_representations, axis=1)
#     scores /= item_norms

#     best = np.argpartition(scores, -N)[-N:]
#     return sorted(zip(best, scores[best] / item_norms[item_id]), 
#                   key=lambda x: -x[1])